# 分布式训练
* 将数据与模型 分布到 单机多卡与多机多卡上

In [19]:
import torch

# 设备数
devie_count = torch.cuda.device_count()
# 获取某个设备
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# 迁移模型或数据
data=torch.ones((3,3))
print(data.device)

device = torch.device('cuda:0')
# data_gpu = data.to(device)
# print(data_gpu.device)

net = torch.nn.Sequential(torch.nn.Linear(3,3))
net.to(device)

cpu


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## 单机多卡
`torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)`
* model 模型
* device_ids GPU设备号
* output_device 输出结果的设备，默认为0，第一块卡

在模型推理过程中，数据被划分多个块，推送到不同的GPU，但是模型在每个GPU都会复制一份
```
class ASimpleNet(nn.Module):
    def __init__(self, layers=3):
        super(ASimpleNet, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(3, 3, bias=False) for i in range(layers)])
    def forward(self, x):
        print("forward batchsize is: {}".format(x.size()[0]))
        x = self.linears(x)
        x = torch.relu(x)
        return x
        
batch_size = 16
inputs = torch.randn(batch_size, 3)
labels = torch.randn(batch_size, 3)
inputs, labels = inputs.to(device), labels.to(device)
net = ASimpleNet()
net = nn.DataParallel(net)
net.to(device)
print("CUDA_VISIBLE_DEVICES :{}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

for epoch in range(1):
    outputs = net(inputs)

# Get:
# CUDA_VISIBLE_DEVICES : 3, 2, 1, 0
# forward batchsize is: 4
# forward batchsize is: 4
# forward batchsize is: 4
# forward batchsize is: 4

```